In [1]:
import tensorflow as tf

PARAMS = {
    'pcg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/pcg/balanced', 
    'ecg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/ecg/balanced',
    'modelsave_dir':'/home/ubuntu/intelliscope/models',
    'modelload_pcg_dir':'/home/ubuntu/intelliscope/models/Experiment(PCG-7).h5', #name of the models ###
    'modelload_ecg_dir':'/home/ubuntu/intelliscope/models/Experiment(ECG-7).h5', ###
    'num_epochs': 10,
    'validation_split':0.1,
    'test_split': 0.2,
    'batch_size':32,
    'data_balanced':True,
    'wavelet':'comb1',
    'dataset':'physionet2016a',
    'pcg/ecg': 'pcg+ecg',
    'augmentation':0,
    'opt':tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    'tags':['CASS'],
    'pretrained': False, ###
    'notebook_name':'hybrid01_balanced.ipynb', ###
    'description':'initial model architecture from CASS paper (base model)',
    'name':'data_a + balanced',
    'dropout':0.2,
    'is_features_normalized':False
    }


pcg_dir=PARAMS['pcg_dir']
ecg_dir=PARAMS['ecg_dir']
modelsave_dir=PARAMS['modelsave_dir']
modelload_pcg_dir=PARAMS['modelload_pcg_dir']
modelload_ecg_dir=PARAMS['modelload_ecg_dir']
batch_size=PARAMS['batch_size']

In [2]:
import os
print('check ecg vs pcg compability :: all must be True')
print(os.listdir(PARAMS['pcg_dir']+'/train/0') == os.listdir(PARAMS['ecg_dir']+'/train/0'))
print(os.listdir(PARAMS['pcg_dir']+'/train/1') == os.listdir(PARAMS['ecg_dir']+'/train/1'))
print(os.listdir(PARAMS['pcg_dir']+'/val/0')   == os.listdir(PARAMS['ecg_dir']+'/val/0'))
print(os.listdir(PARAMS['pcg_dir']+'/val/1')   == os.listdir(PARAMS['ecg_dir']+'/val/1'))
print(os.listdir(PARAMS['pcg_dir']+'/test/0')  == os.listdir(PARAMS['ecg_dir']+'/test/0'))
print(os.listdir(PARAMS['pcg_dir']+'/test/1')  == os.listdir(PARAMS['ecg_dir']+'/test/1'))

check ecg vs pcg compability :: all must be True
True
True
True
True
True
True


In [3]:
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization

from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def generate_generator_multiple(generator,dir1, dir2, batch_size):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size=(272, 462),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size=(272, 462),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    while True:
            X1i, y1i = genX1.next()
            X2i, y2i = genX2.next()
                
            yield [X1i, X2i], y1i
    
train_datagen = ImageDataGenerator(rescale=1./255)   
val_datagen = ImageDataGenerator(rescale=1./255)   

dual_training   = generate_generator_multiple(train_datagen,PARAMS['pcg_dir']+'/train/',PARAMS['ecg_dir']+'/train/',batch_size) #1. PCG, 2. ECG
dual_validation = generate_generator_multiple(val_datagen,PARAMS['pcg_dir']+'/val/',PARAMS['ecg_dir']+'/val/',batch_size) #1. PCG, 2. ECG

In [4]:
from intelliscope import get_base_model

if PARAMS['pretrained']==True:
    pcg_model=load_model(modelload_pcg_dir)
    ecg_model=load_model(modelload_ecg_dir)
else:
    pcg_model=get_base_model()
    ecg_model=get_base_model()
    
pcg_layer=[layer.name for layer in pcg_model.layers][-4]
ecg_layer=[layer.name for layer in ecg_model.layers][-4]

pcg_block= tf.keras.Model(inputs=pcg_model.input, outputs=pcg_model.get_layer(pcg_layer).output)
pcg_block.trainable=True
pcg_input = tf.keras.layers.Input(shape=(272,462,3))
pcg_features = pcg_block(pcg_input)

ecg_block= tf.keras.Model(inputs=ecg_model.input, outputs=ecg_model.get_layer(ecg_layer).output)
ecg_block.trainable=True
ecg_input = tf.keras.layers.Input(shape=(272,462,3))
ecg_features = ecg_block(ecg_input)

if PARAMS['is_features_normalized']==True:
    pcg_features = tf.keras.layers.LayerNormalization()(pcg_features)
    ecg_features = tf.keras.layers.LayerNormalization()(ecg_features)
    
concat     = tf.keras.layers.concatenate([pcg_features, ecg_features])
dropout    = tf.keras.layers.Dropout(PARAMS['dropout'])(concat)
dense      = tf.keras.layers.Dense(80, activation='relu')(dropout)
dense_1    = tf.keras.layers.Dense(20, activation='relu')(dense)
output = tf.keras.layers.Dense(2, activation='softmax')(dense_1)

model= tf.keras.Model(inputs=[pcg_input, ecg_input], outputs=[output])



model.compile(PARAMS['opt'],
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])

    

In [5]:
history=model.fit_generator(
        dual_training,
        steps_per_epoch=45,
        epochs=PARAMS['num_epochs'],
        validation_steps=7, 
        validation_data=dual_validation)

Instructions for updating:
Please use Model.fit, which supports generators.
Found 1428 images belonging to 2 classes.
Found 1428 images belonging to 2 classes.


/home/ubuntu/core-env/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


  ...
    to  
  ['...']
Found 194 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
  ...
    to  
  ['...']
Train for 45 steps, validate for 7 steps
Epoch 1/10
45/45 [==============================] - 84s 2s/step - loss: 33.6039 - categorical_accuracy: 0.6366 - val_loss: 0.6909 - val_categorical_accuracy: 0.4794
Epoch 2/10
45/45 [==============================] - 83s 2s/step - loss: 0.7096 - categorical_accuracy: 0.3943 - val_loss: 0.6886 - val_categorical_accuracy: 0.4794
Epoch 3/10
45/45 [==============================] - 83s 2s/step - loss: 0.6465 - categorical_accuracy: 0.4335 - val_loss: 0.9968 - val_categorical_accuracy: 0.4794
Epoch 4/10
45/45 [==============================] - 83s 2s/step - loss: 0.8647 - categorical_accuracy: 0.4174 - val_loss: 0.6919 - val_categorical_accuracy: 0.4794
Epoch 5/10
45/45 [==============================] - 83s 2s/step - loss: 0.7416 - categorical_accuracy: 0.3312 - val_loss: 0.6891 - val_categorical_accuracy: 0.4794
Epoch 

In [6]:
#!pip install neptune-client
import os
import neptune
NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
os.environ['NEPTUNE_PROJECT']="intelliscope/HYBRID"

neptune.init('intelliscope/HYBRID',NEPTUNE_API_TOKEN)
exp = neptune.create_experiment(name=PARAMS['name'],description=PARAMS['description'],params=PARAMS,upload_source_files=PARAMS['notebook_name'],tags=PARAMS['tags'],upload_stdout=True)

model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
model.save(f'{modelsave_dir}/{str(exp)}.h5')
neptune.log_artifact(f'{modelsave_dir}/{str(exp)}.h5')

for i in range(len(history.history['loss'])):
    neptune.log_metric('loss',history.history['loss'][i])
    neptune.log_metric('val loss',history.history['val_loss'][i])
    neptune.log_metric('categorical_acc',history.history['categorical_accuracy'][i])
    neptune.log_metric('val_categorical_acc',history.history['val_categorical_accuracy'][i])

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/HYBRID/e/HYBRID-2


In [9]:
import numpy as np
from neptunecontrib.monitoring.metrics import *

assert len(os.listdir(ecg_dir+'/test/1'))+len(os.listdir(ecg_dir+'/test/0'))== len(os.listdir(pcg_dir+'/test/1'))+len(os.listdir(pcg_dir+'/test/0'))

test_datagen = ImageDataGenerator(rescale=1./255)
test_len=len(os.listdir(ecg_dir+'/test/1'))+len(os.listdir(ecg_dir+'/test/0'))

test_gen_pcg= test_datagen.flow_from_directory(PARAMS['pcg_dir']+'/test',
                                          target_size=(272, 462),
                                          class_mode = 'categorical',
                                          batch_size = test_len,
                                          shuffle=False, 
                                          seed=7)
    
test_gen_ecg = test_datagen.flow_from_directory(PARAMS['ecg_dir']+'/test',
                                          target_size=(272, 462),
                                          class_mode = 'categorical',
                                          batch_size = test_len,
                                          shuffle=False, 
                                          seed=7)


pcg, y1=test_gen_pcg.next()
ecg, y2=test_gen_ecg.next()
assert (y1==y2).all()

prediction = model.predict([pcg, ecg])

y_test=np.argmax(y1,axis=1)
y_pred=prediction


threshold = 0.5

log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
log_classification_report(y_test, y_pred[:, 1] > threshold)
log_class_metrics(y_test, y_pred[:, 1] > threshold)
log_class_metrics_by_threshold(y_test, y_pred[:, 1])
log_brier_loss(y_test, y_pred[:, 1])
log_prediction_distribution(y_test, y_pred[:, 1])

log_log_loss(y_test, y_pred)
log_roc_auc(y_test, y_pred)
log_precision_recall_auc(y_test, y_pred)
log_ks_statistic(y_test, y_pred)
log_cumulative_gain(y_test, y_pred)
log_lift_curve(y_test, y_pred)

Found 388 images belonging to 2 classes.
Found 388 images belonging to 2 classes.


Invalid metric value: nan for channel negative_predictive_value. Metrics with nan or +/-inf values will not be sent to server


In [10]:
neptune.stop()